In [ ]:
from MySQL import MySQL_Run

MySQL_Run("show tables")

# Create Table

In [9]:
MySQL_Run("""
CREATE TABLE IF NOT EXISTS `bus_route_stations` (
  `route_id` INT UNSIGNED NOT NULL COMMENT '所屬路徑ID（對應 bus_routes_total.route_id）',
  `route_name` VARCHAR(255) NOT NULL COMMENT '路徑名稱',
  `direction` ENUM('去程','回程') NOT NULL COMMENT '路程方向',
  `stop_name` VARCHAR(255) NOT NULL COMMENT '站點名稱',
  `latitude` DOUBLE NOT NULL COMMENT '緯度',
  `longitude` DOUBLE NOT NULL COMMENT '經度',
  `eta_from_start` INT UNSIGNED DEFAULT NULL COMMENT '首站到此站的預估時間（分鐘）',
  `stop_order` INT UNSIGNED NOT NULL COMMENT '第幾站（排序用）',
  `address` VARCHAR(255) DEFAULT NULL COMMENT '站點地址',
  `status` TINYINT NOT NULL DEFAULT 0 COMMENT '狀態（0=正常/啟用，可自訂）',
  `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '建立時間',
  KEY `idx_route_id` (`route_id`),
  KEY `idx_route_name` (`route_name`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;

""")

'Query executed.'

In [ ]:
MySQL_Run("""
CREATE TABLE IF NOT EXISTS `bus_routes_total` (
  `route_id` INT UNSIGNED NOT NULL AUTO_INCREMENT COMMENT '路徑ID（自動遞增、唯一）',
  `route_name` VARCHAR(255) NOT NULL COMMENT '路徑名稱',
  `stop_count` INT UNSIGNED NOT NULL COMMENT '站點數量',
  `direction` ENUM('單向', '雙向') NOT NULL COMMENT '單程或雙向',
  `start_stop` VARCHAR(255) DEFAULT NULL COMMENT '首站',
  `end_stop` VARCHAR(255) DEFAULT NULL COMMENT '末站',
  `status` TINYINT NOT NULL DEFAULT 0 COMMENT '狀態（0=正常/啟用，可自訂）',
  `created_at` TIMESTAMP NOT NULL DEFAULT CURRENT_TIMESTAMP COMMENT '建立時間',
  PRIMARY KEY (`route_id`),
  UNIQUE KEY `uq_route_name` (`route_name`)
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_unicode_ci;
""")
MySQL_Run("""
INSERT INTO `bus_routes_total` (`route_name`, `stop_count`, `direction`, `start_stop`, `end_stop`, `status`)
VALUES
  ('市民小巴5(洽公直達線)', 8, '雙向', '花蓮轉運站', '花蓮縣政府 正門', 0),
  ('市民小巴6(醫療照護線)', 8, '雙向', '花蓮轉運站', '門諾醫院', 0),
  ('市民小巴7(市區夜環線) 單向循環線', 11, '單向', '花蓮轉運站', '花蓮轉運站', 0),
  ('市民小巴-行動遊花蓮 單向循環線', 24, '單向', '花蓮轉運站', '七星柴魚博物館', 0);
""")

'Query executed.'

# Instert

In [ ]:
import pandas as pd

df = pd.read_csv(r"D:\2025Cody\Github\H_Bus\client\src\data\Station.csv")
values = []
for _, row in df.iterrows():
    sql = f"({int(row['路徑ID'])}, '{row['路徑名稱']}', '{row['路程']}', '{row['站點']}', {row['去程緯度']}, {row['去程經度']}, {int(row['首站到此站時間'])}, {int(row['站次'])}, '{row['位置']}', {int(row['狀態'])})"
    values.append(sql)

insert_sql = "INSERT INTO `bus_route_stations` (`route_id`, `route_name`, `direction`, `stop_name`, `latitude`, `longitude`, `eta_from_start`, `stop_order`, `address`, `status`) VALUES\n" + ",\n".join(values) + ";"

print(insert_sql)
MySQL_Run(insert_sql)

# Read


In [20]:
Result = MySQL_Run("select * from bus_routes_total")

Result

((1,
  '市民小巴5(洽公直達線)',
  8,
  '雙向',
  '花蓮轉運站',
  '花蓮縣政府 正門',
  0,
  datetime.datetime(2025, 9, 1, 6, 12, 25)),
 (2,
  '市民小巴6(醫療照護線)',
  8,
  '雙向',
  '花蓮轉運站',
  '門諾醫院',
  0,
  datetime.datetime(2025, 9, 1, 6, 12, 25)),
 (3,
  '市民小巴7(市區夜環線) 單向循環線',
  11,
  '單向',
  '花蓮轉運站',
  '花蓮轉運站',
  0,
  datetime.datetime(2025, 9, 1, 6, 12, 25)),
 (4,
  '市民小巴-行動遊花蓮 單向循環線',
  24,
  '單向',
  '花蓮轉運站',
  '七星柴魚博物館',
  0,
  datetime.datetime(2025, 9, 1, 6, 12, 25)))

In [22]:
import pandas as pd 
Result = MySQL_Run("select * from bus_routes_total")
COLUMNS = MySQL_Run("SHOW COLUMNS FROM bus_routes_total")
cols = [c[0] for c in COLUMNS]
df = pd.DataFrame(Result, columns=cols)
df

,route_id,route_name,stop_count,direction,start_stop,end_stop,status,created_at
0,1,市民小巴5(洽公直達線),8,雙向,花蓮轉運站,花蓮縣政府 正門,0,2025-09-01 06:12:25
1,2,市民小巴6(醫療照護線),8,雙向,花蓮轉運站,門諾醫院,0,2025-09-01 06:12:25
2,3,市民小巴7(市區夜環線) 單向循環線,11,單向,花蓮轉運站,花蓮轉運站,0,2025-09-01 06:12:25


In [21]:
MySQL_Run("""DELETE FROM bus_routes_total
WHERE route_name LIKE '%行動遊花蓮%';
""")

'Query executed.'

In [17]:
Result = MySQL_Run("select * from bus_route_stations where route_id = 1")
COLUMNS = MySQL_Run("SHOW COLUMNS FROM bus_route_stations")
cols = [c[0] for c in COLUMNS]
df = pd.DataFrame(Result, columns=cols)
df

,route_id,route_name,direction,stop_name,latitude,longitude,eta_from_start,stop_order,address,status,created_at
0,1,市民小巴5(洽公直達線),去程,花蓮轉運站,23.993020,121.603219,0,1,花蓮縣花蓮市國聯一路98號,0,2025-09-01 08:47:42
1,1,市民小巴5(洽公直達線),去程,花蓮縣動防所,23.992912,121.617817,5,2,花蓮縣花蓮市瑞美路5號,0,2025-09-01 08:47:42
2,1,市民小巴5(洽公直達線),去程,花蓮縣稅務局,23.992744,121.619466,10,3,花蓮縣花蓮市府後路26號前,0,2025-09-01 08:47:42
3,1,市民小巴5(洽公直達線),去程,花蓮縣政府 後門,23.991411,121.619984,15,4,花蓮縣花蓮市府後路15號,0,2025-09-01 08:47:42
4,1,市民小巴5(洽公直達線),去程,花蓮縣地方法院,23.989522,121.619688,20,5,花蓮縣花蓮市府前路15號,0,2025-09-01 08:47:42
5,1,市民小巴5(洽公直達線),去程,婦幼親創園區,23.988424,121.618438,25,6,花蓮縣花蓮市府後路2號,0,2025-09-01 08:47:42
6,1,市民小巴5(洽公直達線),去程,花蓮縣衛生局,23.991812,121.616885,30,7,花蓮縣花蓮市新興路200號,0,2025-09-01 08:47:42
7,1,市民小巴5(洽公直達線),去程,花蓮縣政府 正門,23.991411,121.619984,35,8,花蓮縣花蓮市國聯一路98號,0,2025-09-01 08:47:42
8,1,市民小巴5(洽公直達線),回程,花蓮縣政府 正門,23.991411,121.619984,0,1,花蓮縣花蓮市府前路17號,0,2025-09-01 08:47:42
9,1,市民小巴5(洽公直達線),回程,花蓮縣稅務局,23.992815,121.619477,5,2,花蓮縣花蓮市府後路26號前,0,2025-09-01 08:47:42
